In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision
from torch.optim import SGD
from torch.utils.data import SubsetRandomSampler, DataLoader
from torchvision.transforms import *

data = torchvision.datasets.CIFAR10(".", download=True, train=True)
ds_label: str = "CIFAR"
mean = (data.data / 255).mean()
std = (data.data / 255).std()
data.transform = Compose([Lambda(lambda x: x.convert('L')), ToTensor(), Normalize([mean], [std], True)])


class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, kernel_size=5)
        self.conv2 = nn.Conv2d(30, 60, kernel_size=5)
        self.conv3 = nn.Conv2d(60, 120, kernel_size=3)
        self.fc1 = nn.Linear(3*3*120, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = f.relu(f.max_pool2d(self.conv1(x), 2))
        x = f.relu(f.max_pool2d(self.conv2(x), 2))
        x = f.relu(self.conv3(x))
        x = x.view(-1,3*3*120)
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        return f.log_softmax(x, dim=1)


batch_size: int = 64
epoch: int = 40
lr: float = 0.01
test_split: float = 0.2

random_seed: int = 10
shuffle_dataset: bool = True


dataset_size: int = len(data)
indices: list = list(range(dataset_size))
split: int = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

train_sampler: SubsetRandomSampler = SubsetRandomSampler(train_indices)
test_sampler: SubsetRandomSampler = SubsetRandomSampler(test_indices)

train_loader: DataLoader = DataLoader(data, batch_size=batch_size, sampler=train_sampler)
test_loader: DataLoader = DataLoader(data, batch_size=batch_size, sampler=test_sampler)

model: Network = Network().cuda()

optimizer: SGD = SGD(model.parameters(), lr)

Files already downloaded and verified


In [20]:
for e in range(epoch):
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        result: torch.Tensor = model(x.cuda())
        loss: torch.Tensor = f.cross_entropy(result, y.cuda())
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct: int = 0
        for i, (x, y) in enumerate(test_loader):
            output: torch.Tensor = model(x.cuda())
            correct += torch.sum(output.data.max(1)[1] == y.cuda()).item()
    print(f"\nTest accuracy on epoch {e}: {correct / split}")

torch.save(model.state_dict(), 'model{}.pth'.format(ds_label))



Test accuracy on epoch 0: 0.2215

Test accuracy on epoch 1: 0.2718

Test accuracy on epoch 2: 0.3632

Test accuracy on epoch 3: 0.405

Test accuracy on epoch 4: 0.4433

Test accuracy on epoch 5: 0.4603

Test accuracy on epoch 6: 0.4623

Test accuracy on epoch 7: 0.502

Test accuracy on epoch 8: 0.5119

Test accuracy on epoch 9: 0.5298

Test accuracy on epoch 10: 0.5542

Test accuracy on epoch 11: 0.5483

Test accuracy on epoch 12: 0.5723

Test accuracy on epoch 13: 0.5962

Test accuracy on epoch 14: 0.6005

Test accuracy on epoch 15: 0.5974

Test accuracy on epoch 16: 0.6115

Test accuracy on epoch 17: 0.6232

Test accuracy on epoch 18: 0.6122

Test accuracy on epoch 19: 0.6423

Test accuracy on epoch 20: 0.6544

Test accuracy on epoch 21: 0.6589

Test accuracy on epoch 22: 0.6484

Test accuracy on epoch 23: 0.6604

Test accuracy on epoch 24: 0.6591

Test accuracy on epoch 25: 0.6716

Test accuracy on epoch 26: 0.6746

Test accuracy on epoch 27: 0.6801

Test accuracy on epoch 28: 0.66

In [21]:
from PIL import ImageDraw, Image
import random
new_data = torchvision.datasets.CIFAR10(".", download=True, train=False, transform=None)
new_data_loader: DataLoader = DataLoader(new_data, 1)
file = open('labels{}.csv'.format(ds_label), 'w')
for i, data in enumerate(new_data):
    random_x, random_y = random.randint(0,18), random.randint(0,18)
    img: Image.Image = data[0].copy()
    img_draw: ImageDraw.ImageDraw = ImageDraw.Draw(img)
    img_draw.rectangle([random_x, random_y, random_x + 13, random_y + 13], fill=0)
    img.convert('L').save('{}/img/{}.jpg'.format(ds_label, i))
    file.write('{},{},{},{}\n'.format(i, data[1], random_x, random_y))
file.close()

Files already downloaded and verified


In [3]:
from PIL import Image
from torch.utils.data import TensorDataset
from torchvision.transforms.functional import *
images: list = list()
train_images_ae: list = list()
test_images_ae: list = list()
train_images_misgan: list = list()
test_images_misgan: list = list()
for i in range(10_000):
    img: Image = Image.open('{}/img/{}.jpg'.format(ds_label, i))
    images.append(normalize(to_tensor(img), [mean], [std]))
for i in range(8_000):
    img: Image = Image.open('{}/imgAEtrain/{}.jpg'.format(ds_label, i))
    train_images_ae.append(normalize(to_tensor(img), [mean], [std]))
for i in range(2_000):
    img: Image = Image.open('{}/imgAEtest/{}.jpg'.format(ds_label, i + 8000))
    test_images_ae.append(normalize(to_tensor(img), [mean], [std]))
for i in range(8_000):
    img: Image = Image.open('{}/imgMISGANtrain/{}.jpg'.format(ds_label, i))
    train_images_misgan.append(normalize(to_tensor(img), [mean], [std]))
for i in range(2_000):
    img: Image = Image.open('{}/imgMISGANtest/{}.jpg'.format(ds_label, i + 8000))
    test_images_misgan.append(normalize(to_tensor(img), [mean], [std]))
labels: list = list()
labels_file = open('labels{}.csv'.format(ds_label), 'r')
for i, line in enumerate(labels_file):
    labels.append(int(line.split(',')[1]))
dataset: TensorDataset = TensorDataset(torch.stack(images), torch.LongTensor(labels))
data_loader: DataLoader = DataLoader(dataset, batch_size)
train_loader_ae: DataLoader = DataLoader(TensorDataset(torch.stack(train_images_ae), torch.LongTensor(labels[:8000])), batch_size)
test_loader_ae: DataLoader = DataLoader(TensorDataset(torch.stack(test_images_ae), torch.LongTensor(labels[8000:])), batch_size)
train_loader_misgan: DataLoader = DataLoader(TensorDataset(torch.stack(train_images_misgan), torch.LongTensor(labels[:8000])), batch_size)
test_loader_misgan: DataLoader = DataLoader(TensorDataset(torch.stack(test_images_misgan), torch.LongTensor(labels[8000:])), batch_size)

In [14]:
model = Network()
model.load_state_dict(torch.load('model{}.pth'.format(ds_label)))
model.eval()
def show_accuracy(data_loader, size):
    correct: int = 0
    for i, (x, y) in enumerate(data_loader):
        output: torch.Tensor = model(x)
        correct += torch.sum(output.data.max(1)[1] == y).item()
#         if i == 0:
#             print(output.data.max(1)[1])
#             print(y)
    print(f"\nAccuracy {correct / size}")
show_accuracy(data_loader, 10_000)
show_accuracy(train_loader_ae, 8_000)
show_accuracy(test_loader_ae, 2_000)
show_accuracy(train_loader_misgan, 8_000)
show_accuracy(test_loader_misgan, 2_000)


Accuracy 0.293

Accuracy 0.176

Accuracy 0.1715

Accuracy 0.501625

Accuracy 0.458


In [9]:
def generate_mask(x, y, size):
    return np.concatenate([32 * i + np.arange(x, x + size) for i in
                           np.arange(y, y + size)], axis=0).astype(np.int32)


def data_with_defined_mask(x, mask):
    for i in range(x.shape[0]):
        x[i, generate_mask(mask[i][0], mask[i][1], 12)] = np.nan
    return x

In [47]:
from sklearn.impute import SimpleImputer
images = list()
for i in range(10_000):
    img: Image = Image.open('{}/img/{}.jpg'.format(ds_label,i))
    images.append(np.array(img))
images = np.array(images)
masks = list()
labels_file = open('labels{}.csv'.format(ds_label), 'r')
for line in labels_file:
    line_list: list = line.split(',')
    masks.append((int(line_list[2]), int(line_list[3])))

x, y = images / 255.0, labels

data_train = x.reshape(-1, 32 * 32)

data_train = data_with_defined_mask(data_train, masks)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
data_mean = imp.fit_transform(data_train)

In [48]:
data_mean_tensor = list()
for img in data_mean:
    data_mean_tensor.append(normalize(to_tensor(img.reshape(32, 32)).float(), [mean], [std]))
data_mean_loader: DataLoader = DataLoader(TensorDataset(torch.stack(data_mean_tensor), torch.LongTensor(labels)), batch_size)
show_accuracy(data_mean_loader, 10_000)


Accuracy 0.2938
